<a href="https://colab.research.google.com/github/jiku100/ShadowDetectionMethod/blob/main/DeepLearning/MTMT/MTMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. train


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r "/content/gdrive/My Drive/MTMT" .
%cd MTMT

/content/MTMT


In [ ]:
%cd MTMT

/content/MTMT


In [ ]:
!unzip UnionTrain.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: UnionTrain/ShadowImages/USR_shadow_1532.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1533.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1534.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1535.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1536.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1537.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1538.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1539.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1540.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1541.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1542.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1543.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1544.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1545.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1546.jpg  
  inflating: UnionTrain/ShadowImages/USR_shadow_1547.jpg  
  inflating: UnionTr

In [ ]:
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-y9o4qgyo
  Running command git clone -q https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-y9o4qgyo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pydensecrf: filename=pydensecrf-1.0rc2-cp37-cp37m-linux_x86_64.whl size=2774043 sha256=1c46ac9f19f635b74d49210a9fac9bc1db7aa7d809ee36c693ef698108a4bf21
  Stored in directory: /tmp/pip-ephem-wheel-cache-0m4y9n82/wheels/81/ac/ed/09f1890b954ca1ff7f189518cc9a319138e50e7ea88937fae9
Successfully built pydensecrf


In [ ]:
%cd MTMT

/content/MTMT


In [ ]:
import os
import sys
from tqdm import tqdm
import shutil
import argparse
import logging
import time
import random
import numpy as np

import torch
from torch import nn
import torch.optim as optim
from torchvision import transforms
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import torchvision.utils as vutils
import matplotlib.pyplot as plt

from networks.MTMT import build_model
from utils import ramps, losses
from dataloaders.SBU import SBU, relabel_dataset
from dataloaders import joint_transforms_edge as joint_transforms
from utils.util import AverageMeter, TwoStreamBatchSampler

cudnn.benchmark = True

torch.cuda.set_device(0)
args = {
    'root_path' : './UnionTrain',
    'max_iterations': 10000,
    'batch_size': 6,
    'labeled_bs': 4,
    'base_lr': 0.005,
    'lr_decay': 0.9,
    'edge': 10,
    'deterministic': 0,
    'seed': 1337,
    'gpu': '1',
    'ema_decay' : 0.99,
    'consistency_type' : 'mse',
    'consistency' : 1,
    'consistency_rampup' : 7.0,
    'scale' : 416,
    'subitizing' : 1,
    'repeat' : 3
}

os.environ['CUDA_VISIBLE_DEVICES'] = args['gpu']
batch_size = args['batch_size'] * len(args['gpu'].split(','))
max_iterations = args['max_iterations']
base_lr = args['base_lr']
labeled_bs = args['labeled_bs']
lr_decay = args['lr_decay']
loss_record = 0

train_data_path = args['root_path']
snapshot_path = "./ckpt/"
tmp_path = 'tmp_see'
if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)

if args['deterministic']:
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(args['seed'])
    np.random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    torch.cuda.manual_seed(args['seed'])
else:
    cudnn.benchmark = True

num_classes = 2

def get_current_consistency_weight(epoch):
    # Consistency ramp-up from https://arxiv.org/abs/1610.02242
    return args['consistency'] * ramps.sigmoid_rampup(epoch, args['consistency_rampup'])

def update_ema_variables(model, ema_model, alpha, global_step):
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)

def create_model(ema=False):
    # Network definition
    if ema:
        net = build_model(ema=True)
        net_cuda = net.cuda()
        for param in net_cuda.parameters():
            param.detach_()
    else:
        net = build_model()
        net_cuda = net.cuda()

    return net_cuda

if __name__ == "__main__":
    ## make logger file
    if not os.path.exists(snapshot_path):
        os.makedirs(snapshot_path)

    logging.basicConfig(filename=snapshot_path+"/log.txt", level=logging.INFO,
                        format='[%(asctime)s.%(msecs)03d] %(message)s', datefmt='%H:%M:%S')
    logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

    model = create_model()
    ema_model = create_model(ema=True)

    joint_transform = joint_transforms.Compose([
        joint_transforms.RandomHorizontallyFlip(),
        joint_transforms.Resize((args['scale'], args['scale']))
    ])
    val_joint_transform = joint_transforms.Compose([
        joint_transforms.Resize((args['scale'], args['scale']))
    ])
    img_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    target_transform = transforms.ToTensor()
    to_pil = transforms.ToPILImage()

    db_train = SBU(root=train_data_path, joint_transform=joint_transform, transform=img_transform, target_transform=target_transform, mod='union', multi_task=True, edge=True)

    labeled_idxs, unlabeled_idxs = relabel_dataset(db_train, edge_able=True)
    batch_sampler = TwoStreamBatchSampler(labeled_idxs, unlabeled_idxs, batch_size, batch_size-labeled_bs)
    def worker_init_fn(worker_id):
        random.seed(args['seed']+worker_id)
    trainloader = DataLoader(db_train, batch_sampler=batch_sampler, num_workers=4, worker_init_fn=worker_init_fn)

    model.train()
    ema_model.train()

    optimizer = optim.SGD([
        {'params': [param for name, param in model.named_parameters() if name[-4:] == 'bias'],
         'lr': 2 * base_lr},
        {'params': [param for name, param in model.named_parameters() if name[-4:] != 'bias'],
         'lr': base_lr, 'weight_decay': 0.0001}
    ], momentum=0.9)

    if args['consistency_type'] == 'mse':
        consistency_criterion = losses.sigmoid_mse_loss
    
    logging.info("{} itertations per epoch".format(len(trainloader)))

    iter_num = 0
    max_epoch = max_iterations//len(trainloader)+1
    lr_ = base_lr
    model.train()

    for epoch_num in tqdm(range(max_epoch), ncols=70):
        time1 = time.time()
        shadow_loss2_record, shadow_con_loss2_record, edge_loss_record, edge_con_loss_record = AverageMeter(), AverageMeter(), AverageMeter(), AverageMeter()
        subitizing_loss_record = AverageMeter()
        # loss2_h2l_record, loss3_h2l_record, loss4_h2l_record = AverageMeter(), AverageMeter(), AverageMeter()
        # loss1_l2h_record, loss2_l2h_record, loss3_l2h_record = AverageMeter(), AverageMeter(), AverageMeter()
        # loss4_l2h_record, consistency_record = AverageMeter(), AverageMeter()
        for i_batch, sampled_batch in enumerate(trainloader):
            time2 = time.time()
            optimizer.param_groups[0]['lr'] = 2 * base_lr * (1 - float(iter_num) / max_iterations
                                                                ) ** lr_decay
            optimizer.param_groups[1]['lr'] = base_lr * (1 - float(iter_num) / max_iterations
                                                            ) ** lr_decay
            # print('fetch data cost {}'.format(time2-time1))
            image_batch, label_batch, edge_batch, number_per_batch = sampled_batch['image'], sampled_batch['label'], sampled_batch['edge'], sampled_batch['number_per']
            image_batch, label_batch, edge_batch, number_per_batch = image_batch.cuda(), label_batch.cuda(), edge_batch.cuda(), number_per_batch.cuda()

            noise = torch.clamp(torch.randn_like(image_batch) * 0.1, -0.2, 0.2)
            ema_inputs = image_batch + noise
            up_edge, up_shadow, up_subitizing, up_shadow_final = model(image_batch)
            with torch.no_grad():
                up_edge_ema, up_shadow_ema, up_subitizing_ema, up_shadow_final_ema = ema_model(ema_inputs)

            ## calculate the loss
            ## subitizing loss
            subitizing_loss = losses.sigmoid_mse_loss(up_subitizing[:labeled_bs], number_per_batch[:labeled_bs])
            subitizing_con_loss = losses.sigmoid_mse_loss(up_subitizing[labeled_bs:], up_subitizing_ema[labeled_bs:])
            ## edge loss
            edge_loss = []
            edge_con_loss = []
            for (ix, ix_ema) in zip(up_edge, up_edge_ema):
                edge_loss.append(losses.bce2d_new(ix[:labeled_bs], edge_batch[:labeled_bs], reduction='mean'))
                edge_con_loss.append(consistency_criterion(ix[labeled_bs:], ix_ema[labeled_bs:]))
            edge_loss = sum(edge_loss)
            edge_con_loss = sum(edge_con_loss)
            shadow_loss1 = []
            shadow_loss2 = []
            shadow_con_loss1 = []
            shadow_con_loss2 = []
            for (ix, ix_ema) in zip(up_shadow, up_shadow_ema):
                shadow_loss1.append(F.binary_cross_entropy_with_logits(ix[:labeled_bs], label_batch[:labeled_bs], reduction='mean'))
                shadow_con_loss1.append(consistency_criterion(ix[labeled_bs:], ix_ema[labeled_bs:]))

            for i in range(4):
              ix = up_shadow_final[i]
              ix_ema = up_shadow_final_ema[i]
              shadow_loss1.append(F.binary_cross_entropy_with_logits(ix[:labeled_bs], label_batch[:labeled_bs], reduction='mean'))
              shadow_con_loss1.append(consistency_criterion(ix[labeled_bs:], ix_ema[labeled_bs:]))
            
            shadow_loss2.append(F.binary_cross_entropy_with_logits(up_shadow_final[-1][:labeled_bs], label_batch[:labeled_bs], reduction='mean'))
            shadow_con_loss2.append(consistency_criterion(up_shadow_final[-1][labeled_bs:], up_shadow_final_ema[-1][labeled_bs:]))

            shadow_loss = sum(shadow_loss1) + sum(shadow_loss2)
            supervised_loss = shadow_loss + edge_loss * args['edge'] + subitizing_loss*args['subitizing']

            consistency_weight = get_current_consistency_weight(epoch_num)
            consistency_loss = consistency_weight * (edge_con_loss + sum(shadow_con_loss1) + sum(shadow_con_loss2) + subitizing_con_loss)

            loss = supervised_loss + consistency_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            update_ema_variables(model, ema_model, args['ema_decay'], iter_num)

            iter_num = iter_num + 1

            # loss_all_record.update(loss.item(), batch_size)
            shadow_loss2_record.update(shadow_loss2[-1].item(), labeled_bs)
            edge_loss_record.update(edge_loss.item(), labeled_bs)
            shadow_con_loss2_record.update(shadow_con_loss2[-1].item(), batch_size-labeled_bs)
            edge_con_loss_record.update(edge_con_loss.item(), batch_size-labeled_bs)
            subitizing_loss_record.update(subitizing_loss, labeled_bs)

            logging.info('iteration %d : shadow_f : %f5 , edge: %f5 , subitizing: %f5, shadow_f_con: %f5  edge_con: %f5 loss_weight: %f5, lr: %f5' %
                         (iter_num, shadow_loss2_record.avg, edge_loss_record.avg, subitizing_loss_record.avg, shadow_con_loss2_record.avg,edge_con_loss_record.avg, consistency_weight, optimizer.param_groups[1]['lr']))
            loss_record = 'iteration %d : shadow_f : %f5 , edge: %f5 , shadow_f_con: %f5  edge_con: %f5 loss_weight: %f5, lr: %f5' % \
                         (iter_num, shadow_loss2_record.avg, edge_loss_record.avg, shadow_con_loss2_record.avg,edge_con_loss_record.avg, consistency_weight, optimizer.param_groups[1]['lr'])

            if iter_num % 200 == 0:
                vutils.save_image(torch.sigmoid(up_shadow_final[-1].data), tmp_path + '/iter%d-d_predict_f.jpg' % iter_num, normalize=True,
                                  padding=0)
                vutils.save_image(torch.sigmoid(up_shadow_final_ema[-1].data),
                                  tmp_path + '/iter%d-e_predict_f.jpg' % iter_num, normalize=True,
                                  padding=0)
                vutils.save_image(torch.sigmoid(up_shadow[-1].data), tmp_path + '/iter%d-c_predict.jpg' % iter_num,
                                  normalize=True,
                                  padding=0)
                vutils.save_image(torch.sigmoid(up_edge[-1].data), tmp_path + '/iter%d-g_edge.jpg' % iter_num,
                                  normalize=True, padding=0)
                vutils.save_image(torch.sigmoid(up_edge_ema[-1].data), tmp_path + '/iter%d-h_edge.jpg' % iter_num,
                                  normalize=True, padding=0)
                vutils.save_image(image_batch.data, tmp_path + '/iter%d-a_shadow-data.jpg' % iter_num, padding=0)
                vutils.save_image(label_batch.data, tmp_path + '/iter%d-b_shadow-target.jpg' % iter_num, padding=0)
                vutils.save_image(edge_batch.data, tmp_path + '/iter%d-f_edge-target.jpg' % iter_num, padding=0)
            if iter_num >= max_iterations:
                break
            time1 = time.time()
        if iter_num >= max_iterations:
            break
    save_mode_path = os.path.join(snapshot_path, 'mytrain'+str(max_iterations)+'.pth')
    # save_mode_path_ema = os.path.join(snapshot_path, 'iter_' + str(max_iterations) + '_ema.pth')
    torch.save(model.state_dict(), save_mode_path)
    # torch.save(ema_model.state_dict(), save_mode_path_ema)
    logging.info("save model to {}".format(save_mode_path))

1021 itertations per epoch


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|                                          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


iteration 1 : shadow_f : 0.8158035 , edge: 0.0174625 , subitizing: 0.1766965, shadow_f_con: 0.0218855  edge_con: 0.0201325 loss_weight: 0.0067385, lr: 0.0050005


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)


iteration 2 : shadow_f : 0.7919595 , edge: 0.0185755 , subitizing: 0.1754415, shadow_f_con: 0.0137835  edge_con: 0.0106325 loss_weight: 0.0067385, lr: 0.0050005
iteration 3 : shadow_f : 0.7817645 , edge: 0.0199795 , subitizing: 0.1967465, shadow_f_con: 0.0104925  edge_con: 0.0074065 loss_weight: 0.0067385, lr: 0.0049995
iteration 4 : shadow_f : 0.7626955 , edge: 0.0216275 , subitizing: 0.1920855, shadow_f_con: 0.0092155  edge_con: 0.0057685 loss_weight: 0.0067385, lr: 0.0049995
iteration 5 : shadow_f : 0.7403655 , edge: 0.0247755 , subitizing: 0.1953945, shadow_f_con: 0.0085485  edge_con: 0.0048555 loss_weight: 0.0067385, lr: 0.0049985
iteration 6 : shadow_f : 0.7170725 , edge: 0.0238415 , subitizing: 0.1901235, shadow_f_con: 0.0085895  edge_con: 0.0042775 loss_weight: 0.0067385, lr: 0.0049985
iteration 7 : shadow_f : 0.6923565 , edge: 0.0242825 , subitizing: 0.1862225, shadow_f_con: 0.0083465  edge_con: 0.0038145 loss_weight: 0.0067385, lr: 0.0049975
iteration 8 : shadow_f : 0.6750555

...

iteration 7148 : shadow_f : 0.1283965 , edge: 0.0195635 , subitizing: 0.0017435, shadow_f_con: 0.0021915  edge_con: 0.0025155 loss_weight: 1.0000005, lr: 0.0016175
iteration 7149 : shadow_f : 0.0968515 , edge: 0.0138275 , subitizing: 0.0015935, shadow_f_con: 0.0025395  edge_con: 0.0028535 loss_weight: 1.0000005, lr: 0.0016175
iteration 7150 : shadow_f : 0.1513615 , edge: 0.0119685 , subitizing: 0.0045785, shadow_f_con: 0.0018175  edge_con: 0.0022415 loss_weight: 1.0000005, lr: 0.0016165
iteration 7151 : shadow_f : 0.1407545 , edge: 0.0113425 , subitizing: 0.0043345, shadow_f_con: 0.0016025  edge_con: 0.0023505 loss_weight: 1.0000005, lr: 0.0016165
iteration 7152 : shadow_f : 0.1298035 , edge: 0.0102555 , subitizing: 0.0037925, shadow_f_con: 0.0014535  edge_con: 0.0023885 loss_weight: 1.0000005, lr: 0.0016155
iteration 7153 : shadow_f : 0.1168625 , edge: 0.0097605 , subitizing: 0.0035635, shadow_f_con: 0.0013175  edge_con: 0.0023095 loss_weight: 1.0000005, lr: 0.0016155
iteration 7154 :

 80%|████████████████████████▊      | 8/10 [1:03:27<15:54, 477.12s/it]

iteration 8169 : shadow_f : 0.0777365 , edge: 0.0058525 , subitizing: 0.0012525, shadow_f_con: 0.0010795  edge_con: 0.0020705 loss_weight: 1.0000005, lr: 0.0010855
iteration 8170 : shadow_f : 0.0698805 , edge: 0.0048495 , subitizing: 0.0014175, shadow_f_con: 0.0007595  edge_con: 0.0018355 loss_weight: 1.0000005, lr: 0.0010855
iteration 8171 : shadow_f : 0.0669625 , edge: 0.0052395 , subitizing: 0.0014525, shadow_f_con: 0.0009995  edge_con: 0.0018815 loss_weight: 1.0000005, lr: 0.0010845
iteration 8172 : shadow_f : 0.0584745 , edge: 0.0051525 , subitizing: 0.0018185, shadow_f_con: 0.0017685  edge_con: 0.0022245 loss_weight: 1.0000005, lr: 0.0010845
iteration 8173 : shadow_f : 0.0552215 , edge: 0.0051075 , subitizing: 0.0032675, shadow_f_con: 0.0017555  edge_con: 0.0022555 loss_weight: 1.0000005, lr: 0.0010835
iteration 8174 : shadow_f : 0.0581545 , edge: 0.0057135 , subitizing: 0.0032575, shadow_f_con: 0.0015885  edge_con: 0.0021255 loss_weight: 1.0000005, lr: 0.0010835
iteration 8175 :

 90%|███████████████████████████▉   | 9/10 [1:11:31<07:59, 479.36s/it]

iteration 9190 : shadow_f : 0.0346735 , edge: 0.0045405 , subitizing: 0.0106005, shadow_f_con: 0.0031415  edge_con: 0.0024015 loss_weight: 1.0000005, lr: 0.0005215
iteration 9191 : shadow_f : 0.0715695 , edge: 0.0071615 , subitizing: 0.0064215, shadow_f_con: 0.0018395  edge_con: 0.0019385 loss_weight: 1.0000005, lr: 0.0005215
iteration 9192 : shadow_f : 0.0828855 , edge: 0.0088675 , subitizing: 0.0047945, shadow_f_con: 0.0013315  edge_con: 0.0015995 loss_weight: 1.0000005, lr: 0.0005205
iteration 9193 : shadow_f : 0.0863615 , edge: 0.0100485 , subitizing: 0.0040345, shadow_f_con: 0.0019425  edge_con: 0.0016825 loss_weight: 1.0000005, lr: 0.0005205
iteration 9194 : shadow_f : 0.0863895 , edge: 0.0102385 , subitizing: 0.0035965, shadow_f_con: 0.0017005  edge_con: 0.0016975 loss_weight: 1.0000005, lr: 0.0005195
iteration 9195 : shadow_f : 0.0827615 , edge: 0.0096455 , subitizing: 0.0033435, shadow_f_con: 0.0016305  edge_con: 0.0016445 loss_weight: 1.0000005, lr: 0.0005185
iteration 9196 :

In [ ]:
from google.colab import files
files.download('/content/MTMT/ckpt/iter10_10000.pth')

# 2. Infer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r "/content/gdrive/My Drive/MTMT" .
%cd MTMT

/content/MTMT


In [ ]:
!unzip test.zip

Archive:  test.zip
   creating: test/
  inflating: test/1.jpg              
  inflating: test/2.jpg              
  inflating: test/3.jpg              
  inflating: test/4.jpg              
  inflating: test/5.jpg              
  inflating: test/6.jpg              
  inflating: test/7.jpg              
  inflating: test/Ground1.jpg        
  inflating: test/Ground10.jpg       
  inflating: test/Ground11.jpg       
  inflating: test/Ground12.jpg       
  inflating: test/Ground13.jpg       
  inflating: test/Ground14.jpg       
  inflating: test/Ground2.jpg        
  inflating: test/Ground3.jpg        
  inflating: test/Ground4.jpg        
  inflating: test/Ground5.jpg        
  inflating: test/Ground6.jpg        
  inflating: test/Ground7.jpg        
  inflating: test/Ground8.jpg        
  inflating: test/Ground9.jpg        


In [ ]:
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-t_hiyexj
  Running command git clone -q https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-t_hiyexj
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pydensecrf: filename=pydensecrf-1.0rc2-cp37-cp37m-linux_x86_64.whl size=2774269 sha256=04ec292b4a8d9d4b23f2bb642d1f301988d578eedddb467f84bd13ce9b6d58ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-oicuvi2u/wheels/81/ac/ed/09f1890b954ca1ff7f189518cc9a319138e50e7ea88937fae9
Successfully built pydensecrf


In [ ]:
%cd MTMT

/content/MTMT


In [ ]:
import numpy as np
import os
import cv2

import torch
from PIL import Image
from torch.autograd import Variable
from torchvision import transforms


from utils.util import crf_refine
from networks.MTMT import build_model
import matplotlib.pyplot as plt

torch.cuda.set_device(0)

ckpt_path = './ckpt'
exp_name = 'MTMT'
args = {
    'scale': 416
}

img_transform = transforms.Compose([
    transforms.Resize((args['scale'], args['scale'])),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

to_test = {'test': "/content/MTMT/test"}

to_pil = transforms.ToPILImage()

loadpath = "/content/gdrive/MyDrive/Weight/mymodel.pth"
if not os.path.exists("./Output"):
  os.mkdir("./Output")

def main():
    net = build_model().cuda()
    net.load_state_dict(torch.load(loadpath))
    net.eval()
    with torch.no_grad():
        for name, root in to_test.items():
            img_list = [img_name for img_name in os.listdir(root) if
                        img_name.endswith('.jpg')]
            for idx, img_name in enumerate(img_list):
                print('predicting for %s: %d / %d' % (name, idx + 1, len(img_list)))
                img = cv2.imread(os.path.join(root, img_name))
                h, w, _  = img.shape
                img = Image.fromarray(img).convert('RGB')
                img_var = img_transform(img)
                img_var = Variable(img_var.unsqueeze(0)).cuda()
                _,_,_,res = net(img_var)
                res = torch.sigmoid(res[-1])
                prediction = np.array(transforms.Resize((h, w))(to_pil(res.data.squeeze(0).cpu())))
                prediction = crf_refine(np.array(img.convert('RGB')), prediction)
                Image.fromarray(prediction).save(
                    os.path.join("/content/MTMT/Output", "Output" + str(idx) + ".jpg"))


if __name__ == '__main__':
    main()

predicting for test: 1 / 21
predicting for test: 2 / 21
predicting for test: 3 / 21
predicting for test: 4 / 21
predicting for test: 5 / 21
predicting for test: 6 / 21
predicting for test: 7 / 21
predicting for test: 8 / 21
predicting for test: 9 / 21
predicting for test: 10 / 21
predicting for test: 11 / 21
predicting for test: 12 / 21
predicting for test: 13 / 21
predicting for test: 14 / 21
predicting for test: 15 / 21
predicting for test: 16 / 21
predicting for test: 17 / 21
predicting for test: 18 / 21
predicting for test: 19 / 21
predicting for test: 20 / 21
predicting for test: 21 / 21


In [ ]:
!zip mymodel_2.zip -r Output

  adding: Output/ (stored 0%)
  adding: Output/Output0.jpg (deflated 1%)
  adding: Output/Output4.jpg (deflated 27%)
  adding: Output/Output17.jpg (deflated 57%)
  adding: Output/Output18.jpg (deflated 19%)
  adding: Output/Output5.jpg (deflated 22%)
  adding: Output/Output14.jpg (deflated 10%)
  adding: Output/Output2.jpg (deflated 7%)
  adding: Output/Output3.jpg (deflated 26%)
  adding: Output/Output10.jpg (deflated 35%)
  adding: Output/Output6.jpg (deflated 16%)
  adding: Output/Output15.jpg (deflated 16%)
  adding: Output/Output7.jpg (deflated 46%)
  adding: Output/Output16.jpg (deflated 55%)
  adding: Output/Output1.jpg (deflated 36%)
  adding: Output/Output13.jpg (deflated 12%)
  adding: Output/Output11.jpg (deflated 25%)
  adding: Output/Output8.jpg (deflated 13%)
  adding: Output/Output19.jpg (deflated 7%)
  adding: Output/Output9.jpg (deflated 12%)
  adding: Output/Output12.jpg (deflated 40%)
  adding: Output/Output20.jpg (deflated 16%)


In [ ]:
from google.colab import files
files.download('./mymodel_2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp ./ckpt/mymodel.pth "/content/gdrive/MyDrive/Weight/mytrain.pth"